# <font color="#747a7f">6</font> Interação

<em>"Um gráfico não é 'desenhado' de uma vez; ele é 'construído' e reconstruído até revelar todos os relacionamentos constituídos pelas relações entre os dados. As melhores operações de gráficos são aquelas feitas pelas decisões deles mesmos." — [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)</em>

A visualização oferece um poderoso método de dar sentido aos dados. Uma única imagem, entretanto, tipicamente oferece respostas para, no melhor dos casos, um punhado de perguntas. Pela *interação* nós podemos transformar imagens estáticas em ferramentas de exploração: destacando pontos de interesse, dando zoom para revelar padrões sutis, e fazer ligações entre visualizações múltiplas para pensar sobre relacionamentos multi-dimensionais.

Nos fundamentos da interação está a noção de uma *seleção*: um método de indicar para o computador quais elementos ou regiões nós estamos interessados. Por exemplo, nós podemos passar o mouse sobre um ponto, clicar em várias marcas, ou desenhar um contorno ao redor de uma região para destacar subconjuntos de dados para investigações mais minuciosas.

Além de codificações visuais e transformações de dados, o Altair oferece uma abstração *selection* para criar interações. Estas seleções encorporam três aspectos:

1. Evento de tratamento de entrada para selecionar pontos ou regiões de interesse, assim como mouse hover, click, drag, scroll, e eventos de toque.
2. Generalizar a partir de uma entrada para formar uma regra de seleção (ou <em>[predicado](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)</em>) que determina se um dado registro está ou não na seleção.
3. Usar o predicado de seleção para configurar dinamicamente uma visualização por meio de *codificações condicionais*, *transformações de filtro*, ou *domínios de escala*.

Este notebook introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, assim como consultas dinâmicas, panning & zooming, details-on-demand, e brushing & linking.

Este notebook é parte do [data visualization curriculum](https://github.com/uwdata/visualization-curriculum).

In [ ]:
import pandas as pd
import altair as alt

## <font color="#747a7f">6.1</font> Datasets

Nós iremos visualizar uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- Um dataset de carros (`cars`) dos anos 70 e início dos anos 80,
- Um dataset de filmes (`movies`), previamente usados no notebook [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- Um dataset contendo dez anos dos preços das ações [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`),
- Um dataset de ações (`stocks`) de empresas de tecnologia, e
- Um dataset de voos (`flights`), incluindo tempo de embarque, distância, e atraso no desembarque.

In [ ]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## <font color="#747a7f">6.2</font> Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando um ponto para destacá-lo. Usando o dataset `cars`, nós iremos começar com um scatter plot de cavalos de potência versos milhas por galão, com uma codificação de cor para o número de cilíndros no motor do carro.

Além disso, nós iremos criar uma instância de seleção chamando `alt.selection_point()`, indicando que nós queremos uma seleção definida sobre um único valor. Por padrão, a seleção usa um mouse click para determinar o valor selecionado. Para registrar uma seleção com o gráfico, nós devemos adicioná-lo usando o método `.add_params()`.

Assim que nossa seleção é definida, nós podemos usá-la como um parâmetro para *codificações condicionais*, as quais aplicam uma codificação diferente a depender se um registro está dentro ou fora da seleção. Por exemplo, considere o seguinte código:

In [ ]:
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))

Esta definição de codificação declara que pontos contidos na `selection` devem ser coloridos de acordo ao campo `Cylinder`, enquanto pontos não-selecionados devem usar um cinza padrão. Uma seleção vazia inclue *todos* os pontos, e então inicialmente todos os pontos irão ser coloridos.

*Tente clicar em diferentes pontos do gráfico abaixo com click e shift-ckick. O que acontece? (Clique no fundo para limpar a declaração da seleção e retornar para uma seleção "vazia".)*

In [ ]:
selection = alt.selection_point();

alt.Chart(cars).mark_circle().add_params(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

alt.Chart(...)

Como nós iremos ver, seleções de pontos oferecem úteis blocos de construção para interações mais poderosas. Ademais, seleções de pontos são somente um dos dois tipos de seleção oferecidos pelo Altair:

- `selection_point` - seleciona múltiplos valores discretos. O primeiro valor é selecionado no mouse click e valores adicionais alternados usando shift-click.
- `selection_interval` - seleciona um intervalo contínuo de valores, iniciado por mouse drag.

Vamos comparar cada um destes tipos de seleção lado a lado. Para manter nosso código organizado nós iremos definir uma função (`plot`) que gera uma especificação de scatter plot exatamente como a função abaixo. Nós podemos passar uma seleção para a função `plot` para tê-la aplicada ao gráfico:

In [ ]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_params(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar duas variantes de gráficos, um para cada tipo de seleção.

O primeiro gráfico (`point`) replica nosso exemplo anterior, suportando interações shift-click para alternar inclusões de múltiplos pontos na seleção. O segundo gráfico (`interval`) gera uma região de seleção (ou *brush*) no mouse drag. Uma vez criada, você pode arrastar o brush ao redor para selecionar diferentes pontos, ou rolar o scroll quando o cursor está dentro do brush para mudar o tamanho do brush (zoom).

*Tente interagir com cada um dos gráficos abaixo!*

In [ ]:
alt.hconcat(
  plot(alt.selection_point()).properties(title='Point (Click e Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

alt.HConcatChart(...)

O exemplo abaixo usa interações padrões (click, shift-click, drag). Nós podemos personalizar mais as interações oferecendo especificações de eventos de entrada usando a [sintáxe do seletor de eventos do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, nós podemos modificar nosso gráfico `point` para agir nos eventos `mouseover` ao invés de eventos `click`.

*Segure a tecla shift no gráfico para "pintar" com dados!*

In [ ]:
alt.hconcat(
  plot(alt.selection_point(on='mouseover')).properties(title='Point (Mouseover e Shift-Mouseover)'),
)

alt.HConcatChart(...)

Agora que nós cobrimos o básico de seleções do Altair, vamos fazer um tour pelas várias técnicas de interação que ele permite!